# Connect to DB

In [1]:
%load_ext sql

In [2]:
import os

In [3]:
host = "localhost"
database = "postgres"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

In [4]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [5]:
from sqlalchemy import create_engine

In [6]:
#Create connection
engine = create_engine(connection_string)

In [7]:
from sqlalchemy import inspect

In [8]:
#Check connection
insp = inspect(engine)
insp.get_table_names()

['spatial_ref_sys',
 'flickr_edin',
 'flickr_castle_points',
 'castle_false_positives',
 'castle_false_negatives',
 'flickrgrid_250m',
 'highest_weight_grid_250m',
 'grid_300m',
 'flickrgrid_300m',
 'grid_250m',
 'tags',
 'tmp_flickredin',
 'highest_weight_grid_300m',
 'castle_positives',
 'tags_per_grid_250m',
 'tags_per_grid_300m',
 'highest_weight_grid_300m_v2',
 'highest_weight_grid_250m_v2']

In [9]:
import pandas as pd

In [10]:
df = pd.read_sql('SELECT * FROM flickr_edin', engine)

In [11]:
#Word count of tags with duplicates
words = []
for x in range(len(df)):
    tags = df.usertags[x]
    removeBar = tags.split("|")
    
    for i in range(len(removeBar)):
        tmp = removeBar[i].split(" ")
        
        for j in range(len(tmp)):
            words.append(tmp[j])

print(len(words))

655062


In [12]:
#Word count of tags without duplicates
words = []
for x in range(len(df)):
    tags = df.usertags[x]
    #print(tags)
    removeBar = tags.split("|")

    for i in range(len(removeBar)):
        tmp = removeBar[i].split(" ")
        
        for j in range(len(tmp)):
            words.append(tmp[j])

#remove duplicates
words = list(dict.fromkeys(words))

print(len(words))

19511


# Create graphs

In [14]:
#Generate bar chart of the number of photos per day
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
from datetime import date
import calendar

#Data from database
data = pd.read_sql('SELECT date_taken, id FROM flickr_edin', engine)
daysNum = []
ids = []

#Add days of week and ids into separate lists
for i in data.index:
    daysNum.append(data.loc[i, 'date_taken'].weekday())
    ids.append(data.loc[i, 'id'])

#Create dataframe with days of week and ids
d = {'Day of week': daysNum, 'Id': ids}
df = pd.DataFrame(d)
dfg = df.groupby('Day of week').count().reset_index()
dfg = dfg.rename(columns={"Id": "Frequency"})

for i in dfg.index:
    dfg['Day of week'] = dfg['Day of week'].replace([i], calendar.day_name[i])
    
#Plot stucture
fig = px.bar(dfg,
             x='Day of week',
             y='Frequency',
             title='Number of photos per day of the week',
             color='Day of week',
             barmode='stack')

#Plot
fig.show()

In [15]:
#Generate a bar chart showing how many images were uploaded each month
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
from datetime import date
import calendar

#Data from database
data = pd.read_sql('SELECT date_taken, id FROM flickr_edin', engine)
monthsNum = []
ids = []

#Add days of week and ids into separate lists
for i in data.index:
    monthsNum.append(data.loc[i, 'date_taken'].month)
    ids.append(data.loc[i, 'id'])

#Create dataframe with days of week and ids
d = {'Month': monthsNum, 'Id': ids}
df = pd.DataFrame(d)
dfg = df.groupby('Month').count().reset_index()
dfg = dfg.rename(columns={"Id": "Frequency"})

for i in dfg.index + 1:
    dfg['Month'] = dfg['Month'].replace([i], calendar.month_name[i])
    
#Plot stucture
fig = px.bar(dfg,
             x='Month',
             y='Frequency',
             title='Number Of Images Uploaded Per Month',
             #color='Month',
             barmode='stack')

#Plot
fig.show()

# Calculate TF-IDF

In [21]:
#Get tags with corressponding row and col - limited user tag usage
tagPerGrid = pd.read_sql('SELECT userid, tag, row, col FROM tags_per_grid_250m GROUP BY tag, userid, row, col ORDER BY row,col', engine)

In [100]:
#Get tags with corressponding row and col - tag occurance
#tagPerGrid = pd.read_sql('SELECT userid, tag, row, col FROM tags_per_grid_250m ORDER BY row,col', engine)

In [22]:
#Remove spaces
tagPerGrid['tag'] = tagPerGrid['tag'].str.replace(' ', '')

In [23]:
#Create dictionary with the grid cell as the key and the value as a list of tags in the grid cell

allTagsInGrid = {} #Create dictionary
firstKey = str(tagPerGrid.loc[0, 'row']) + "," + str(tagPerGrid.loc[0, 'col']) #Get the first key
allTagsInGrid[firstKey] = [] #Add the first key with an empty list as its value to the dictionary

for i in tagPerGrid.index: #Loop for every tag 
    row = tagPerGrid.loc[i, 'row']
    col = tagPerGrid.loc[i, 'col']
    tmpKey = str(row) + "," + str(col) #Create the key from the row and column 
    
    if tmpKey in allTagsInGrid: #If the key exists, append the tag to the list 
        allTagsInGrid[tmpKey].append(tagPerGrid.loc[i, 'tag'])
    else: #If the key does not exist, add the key with an empty list and then append the tag
        allTagsInGrid[tmpKey] = []
        allTagsInGrid[tmpKey].append(tagPerGrid.loc[i, 'tag'])

In [24]:
#Get the unique tags for each grid
uniqueTags = allTagsInGrid.copy()
keys = list(uniqueTags.keys())

for i in uniqueTags:
    tmpTags = uniqueTags[i]
    tmpUnique = list(dict.fromkeys(tmpTags))
    uniqueTags[i] = tmpUnique

In [25]:
#Count how many times each tag occurs in the grid
count = []

for i in uniqueTags:
    tmpUniqueTags = uniqueTags[i]
    tmpAllTags = allTagsInGrid[i]
    for j in tmpUniqueTags:
        count.append(tmpAllTags.count(j))

In [26]:
#Create tags with tagCount as a dataframe
tags = []
grids = []

for i in uniqueTags:
    tmpTags = uniqueTags[i]
    for j in tmpTags:
        tags.append(j)
        grids.append(i)

tmp = {'Word': tags, 'Grid': grids, 'CountInGrid': count}
tagCount = pd.DataFrame(tmp)

In [27]:
#Calculate TF
tf = []
for i in range(len(tagCount)):
    theCount = tagCount.loc[i, 'CountInGrid']
    theGrid = tagCount.loc[i, 'Grid']
    numOfTagsInGrid = len(allTagsInGrid[theGrid])
    
    tf.append(theCount/numOfTagsInGrid)

In [28]:
#Add to dataframe
tagCount['TF'] = tf

In [29]:
#Calculate number of times each tag occurs in a grid
numOfGridsWithTag = [] #Create an emtpy list to store the number of grids a tag occurs in
tags = [] #Create an emtpy list to store all the tags
keys = list(uniqueTags.keys()) #Get the keys from the dictionary 

for i in uniqueTags: #Loop for the entire dictionary 
    tmpTags = uniqueTags[i] #Get the tags with the current key
    for j in tmpTags: #Loop for each tag
        tags.append(j) #Append the tag to the list
        
for i in tags: #loop for each tag
    count = 0 
    for j in keys: #loop for each key
        if i in allTagsInGrid[j]: #Check if tag is in grid
            count += 1 #If the tag is in the grid, add one to the count
    numOfGridsWithTag.append(count) #Append the count to the list


In [30]:
#Add to dataframe
tagCount["CountInAllGrids"] = numOfGridsWithTag

In [31]:
#Calculate IDF
import math
idf = []
totalNumOfGrids = len(uniqueTags)

for i in range(len(numOfGridsWithTag)):
    tmp = math.log(totalNumOfGrids/numOfGridsWithTag[i])
    idf.append(tmp)

In [32]:
#Add to dataframe
tagCount["IDF"] = idf

In [33]:
#Calculate TF-IDF
tfidf = []

for i in range(len(tagCount)):
    tmp = tagCount.loc[i, 'TF'] * tagCount.loc[i, 'IDF']
    tfidf.append(tmp)

In [34]:
#Add to dataframe
tagCount['TF-IDF'] = tfidf

In [35]:
#Get the highest weighted tag for each grid
grids = list(uniqueTags.keys())
tags = []

for i in range(len(grids)):
    tagsInGrid = tagCount[tagCount['Grid']==grids[i]] #get tags in the grid
    sortedTagsInGrid =  tagsInGrid.sort_values(by=['TF-IDF'], ascending=False) #sort the tags in grid by tf-idf
    highestWeight = sortedTagsInGrid.head(1).reset_index() #Get row with highest weight
    tags.append(highestWeight.loc[0,'Word']) #get the tag and append it to the tags list

In [36]:
#Get the grids with their geom
gridsWithGeom = pd.read_sql('SELECT row, col, geom FROM flickrgrid_250m GROUP BY geom, row, col ORDER BY row, col', engine)

In [37]:
#Remove grids which are not in the dataframe (i.e. grids which have a flickr image inside it but does not have a tag)
removeRow = []

for i in range(len(gridsWithGeom)):
    tmpGrid = str(gridsWithGeom.loc[i, 'row']) + "," + str(gridsWithGeom.loc[i, 'col'])
    for j in grids: #Check if the grid is in the list of grids
        if tmpGrid == j:
            contains = True
            break
        else:
            contains = False
    if contains == False: #If the grid is not in the list of grids, add the index to the list
        removeRow.append(i)

gridsWithGeom = gridsWithGeom.drop(gridsWithGeom.index[removeRow]) #Remove rows 

In [38]:
#Add to dataframe
gridsWithGeom['tag']=tags

In [39]:
#Reorder columns
cols = gridsWithGeom.columns.tolist()
cols = cols[-1:] + cols[:-1]
gridsWithGeom = gridsWithGeom[cols]

In [40]:
#Reset index
gridsWithGeom = gridsWithGeom.reset_index()

In [41]:
#Remove index column
gridsWithGeom = gridsWithGeom.drop(columns=['index'])

In [102]:
#Create table in database
gridsWithGeom.to_sql(name = 'highest_weight_grid_250m_v2',
                     con = engine,
                     if_exists='replace',
                     index=False
                    )

895